In [2]:
from pysnmp.entity.rfc3413.oneliner import cmdgen

In [3]:
cmd = cmdgen.CommandGenerator()

In [4]:
errorIndication, errorStatus, errorIndex, varBinds = cmd.getCmd(
    # v1
    # cmdgen.CommunityData('public', 0),
    # v2
    # cmdgen.CommunityData('public'),
    # v3
    cmdgen.UsmUserData(
        'demo', 'my_new_password', 'my_new_password', 
        cmdgen.usmHMACMD5AuthProtocol,
        cmdgen.usmDESPrivProtocol
    ),
    cmdgen.UdpTransportTarget(('ss.laisky.com', 161)),
    cmdgen.MibVariable('1.3.6.1.2.1.1.1.0')
)

print(
    'errorIndication: {}\n'
    'errorStatus: {}\n'
    'errorIndex: {}\n'
    'varBinds: {}'
    .format(errorIndication, errorStatus, errorIndex, varBinds)
)

errorIndication: None
errorStatus: 0
errorIndex: 0
varBinds: [(ObjectName(1.3.6.1.2.1.1.1.0), OctetString('b'Linux li744-109 4.1.5-x86_64-linode61 #7 SMP Mon Aug 24 13:46:31 EDT 2015 x86_64''))]


chexiang
===

- system = 1.3.6.1.2.1.1
- sysName = 1.3.6.1.2.1.1.5
- ifEntry = 1.3.6.1.2.1.2.2.1
- ifDescr = 1.3.6.1.2.1.2.2.1.2
- ifSpeed = 1.3.6.1.2.1.2.2.1.5

In [95]:
import string
from pysnmp.smi import builder, view, error
from pysnmp.entity.rfc3413.oneliner import cmdgen
from collections import namedtuple as NT

cmdGen = cmdgen.CommandGenerator()
mibBuilder = builder.MibBuilder()
mibBuilder.loadModules(
    'RFC-1213', 'RFC-3413'
    )
mibView = view.MibViewController(mibBuilder)
errorIndication, errorStatus, errorIndex, \
                 varBindTable = cmdGen.nextCmd(
    # SNMP v2
    cmdgen.CommunityData('huawei@123'),
    # Transport
    cmdgen.UdpTransportTarget(('10.32.33.10', 161)),
#     cmdgen.MibVariable('IF-MIB', 'ifInOctets'),
    cmdgen.MibVariable('IF-MIB', 'ifDescr'),
)


for row in varBindTable:
    for oid, val in row:
#         print(repr(oid))
        print(oid, val.prettyPrint())

1.3.6.1.2.1.2.2.1.2.1 NULL0
1.3.6.1.2.1.2.2.1.2.2 InLoopBack0
1.3.6.1.2.1.2.2.1.2.3 MEth0/0/0
1.3.6.1.2.1.2.2.1.2.5 GE1/0/1
1.3.6.1.2.1.2.2.1.2.6 GE1/0/2
1.3.6.1.2.1.2.2.1.2.7 GE1/0/3
1.3.6.1.2.1.2.2.1.2.8 GE1/0/4
1.3.6.1.2.1.2.2.1.2.9 GE1/0/5
1.3.6.1.2.1.2.2.1.2.10 GE1/0/6
1.3.6.1.2.1.2.2.1.2.11 GE1/0/7
1.3.6.1.2.1.2.2.1.2.12 GE1/0/8
1.3.6.1.2.1.2.2.1.2.13 GE1/0/9
1.3.6.1.2.1.2.2.1.2.14 GE1/0/10
1.3.6.1.2.1.2.2.1.2.15 GE1/0/11
1.3.6.1.2.1.2.2.1.2.16 GE1/0/12
1.3.6.1.2.1.2.2.1.2.17 GE1/0/13
1.3.6.1.2.1.2.2.1.2.18 GE1/0/14
1.3.6.1.2.1.2.2.1.2.19 GE1/0/15
1.3.6.1.2.1.2.2.1.2.20 GE1/0/16
1.3.6.1.2.1.2.2.1.2.21 GE1/0/17
1.3.6.1.2.1.2.2.1.2.22 GE1/0/18
1.3.6.1.2.1.2.2.1.2.23 GE1/0/19
1.3.6.1.2.1.2.2.1.2.24 GE1/0/20
1.3.6.1.2.1.2.2.1.2.25 GE1/0/21
1.3.6.1.2.1.2.2.1.2.26 GE1/0/22
1.3.6.1.2.1.2.2.1.2.27 GE1/0/23
1.3.6.1.2.1.2.2.1.2.28 GE1/0/24
1.3.6.1.2.1.2.2.1.2.29 GE1/0/25
1.3.6.1.2.1.2.2.1.2.30 GE1/0/26
1.3.6.1.2.1.2.2.1.2.31 GE1/0/27
1.3.6.1.2.1.2.2.1.2.32 GE1/0/28
1.3.6.1.2.1.2.2.1.2.33 G

In [85]:
o = cmdgen.MibVariable('IF-MIB', 'ifInOctets', 1)
o.loadMibs()

ObjectIdentity('IF-MIB', 'ifInOctets', 1)